# Background on `likelihood_inference`

## Preamble

This document will serve as the mathematical background to Estimagic's `likelihood_inference` functionality. The main part concerning `likelihood_inference` is with respect to the section: *Variance Estimation*.

## Introduction

Maximum-likelihood estimation seeks to maximize the likelihood function, defined as a function of an unknown parameter vector, **$\beta$** , conditional on the data provided. For simplicity, we represent such a function as: $$L(\beta|y_j, x_j)$$ where $y_j$ is the observed outcome variable for observation $j$ from the sample data and $x_j$ is the $j$th row of covariates for the same observation. Specifically, the function is called the **joint probability density** since the probability density function for each observation is seen as independent of one another and the distribution from which the observed outcome variable is generated stays the same for each observation [2]; thus, we seek to maximize the product of the densities $$ L(\beta|y_j, x_j) = \prod_{j=1}^{n} f(y_j| x_j, \beta) $$ In practice, independence between observations might not hold or observations are weighted. For these deviations, we maximize the **pseudolikelihood function**, which provides good estimates of the parameters by using an approximation of the true joint probability density. Furthermore, the log of the pseudolikelihood function is used because it is simpler to maximize. The logarithm is a monotonic transformation, thus the parameters that maximize the product of the densities will also maximize the sum of the log-likelihoods. $$\ln L(\beta|y_j, x_j) = \sum^n_{j=1} \ln f(y_j|x_j, \beta) $$ 

The sum of the densities is equal to the product of the densities when observations are independent. Deviation implies the sum is an *approximation* of the true likelihood. The functional form of $f$ depends on the distribution of the disturbance or outcome variable from the estimation model. For the present documentation, we consider the logistical distribution (logit) and the normal distribution (probit). These distirbutions work best for bounded outcome variables, where the probability of observing an outcome lies between 0 and 1 and the sum of the probabilities is equal to 1. 

In [10]:
def estimate_likelihood_obs(params, y, x, criterion, design_options):
    """Pseudo-log-likelihood contribution per individual.

    Args:
        params (pd.DataFrame): The index consists of the parmater names,
            the "value" column are the parameter values.
        y (np.array): 1d numpy array with the dependent variable
        x (np.array): 2d numpy array with the independent variables
        criterion (string): "logit" or "probit"
        design_options (pd.DataFrame): dataframe containing psu, stratum,
            population/design weight and/or a finite population corrector (fpc)
            
    Returns:
        loglike (np.array): 1d numpy array with likelihood contribution per individual

    """
    q = 2 * y - 1
    if criterion=="logit":
        c = np.log(1 / (1 + np.exp(-(q * np.dot(x, params["value"])))))
        if "weight" in design_options.columns:
            return c * design_options["weight"].to_numpy()
        else:
            return c
    elif criterion=="probit":
        c = np.log(stats.norm._cdf(np.dot(q[:, None] * x, params["value"])))
        if "weight" in design_options.columns:
            return c * design_options["weight"].to_numpy()
        else:
            return c
    else:
        print("Criterion function is misspecified or not supported.")

def estimate_likelihood(params, y, x, criterion, design_options):
    """Pseudo-log-likelihood.

    Args:
        params (pd.DataFrame): The index consists of the parameter names,
            the "value" column are the parameter values.
        y (np.array): 1d numpy array with the dependent variable
        x (np.array): 2d numpy array with the independent variables
        criterion (string): "logit" or "probit"
        design_options (pd.DataFrame): dataframe containing psu, stratum,
            population/design weight and/or a finite population corrector (fpc)
            
    Returns:
        loglike (np.array): 1d numpy array with sum of likelihood contributions

    """
    return estimate_likelihood_obs(params, y, x, criterion, design_options).sum()

## Parameter Maximization

To find the vector of parameters which maximize the log-pseudolikelihood function, we take first order conditions of the likelihood with respect to the parameter vector: $$G(\hat{\beta}) = \sum^n_{j=1} \frac{\partial \ln L(\hat{\beta}|y_j, x_j)}{\partial \hat{\beta}} = 0$$ The partial derivative of the log-likelihood for all observations with respect to the vector of parameters makes the **score vector** and we express the sum over all scores as $G(\hat{\beta})$. The existence of a global maximum depends on the shape of the function we are estimating and the parameter space. When the parameter space is compact and the log-likelihood is continuous on that space (i.e. for all parameter values, the log-likelihood returns a value), then a maximum exists. When the parameter space is convex and the log-likelihood f is strictly concave on the set of parameters, then there is a unique set of parameters which maximize the likelihood. 

To generate the parameters which maximize the log-likelihood function, we use Estimagic's `maximize` functionality. For maximum-likelihood estimations, there is not always a closed-form solution; thus, numerical optimization is requisite in estimating the set of parameters. To estimate, there are several methods, specified [here](https://github.com/OpenSourceEconomics/estimagic/blob/master/estimagic/optimization/algo_dict.json). We use the L-BFGS-B algorithm. Running the algorithm returns a set of parameters which maximize the log-pseudolikelihood. 

In [13]:
def estimate_parameters(log_like, design_options, log_like_kwargs, dashboard=False):
    """Estimate parameters that maximize log-likelihood function.

    Args:
        log_like (function): log-likelihood function
        design_options (pd.DataFrame): dataframe containing psu, stratum,
            population/design weight and/or a finite population corrector (fpc)
        log_like_kwargs (dict): contains model equation, data, and type of
            binary-choice model
            Example:
                log_like_kwargs = {
                   "formulas": formulas,
                   "data": orig_data,
                   "model": "logit"
                   }
        dashboard (bool): Switch on the dashboard

    Returns:
        res: parameter vector and other optimization results.

    """
    params, y, x = mle_processing(log_like_kwargs["formulas"], log_like_kwargs["data"])
    res = maximize(
        criterion=log_like,
        params=params,
        algorithm="scipy_L-BFGS-B",
        criterion_kwargs=log_like_kwargs,
        dashboard=dashboard,
    )
    return res

## Variance Estimation

After parameter estimation, we extract the standard errors. To extract the standard-errors, we first solve for the variance-covariance matrix of the parameter vector. There are three methods to estimate the variance: 

(1) Taking the first order Taylor-Series expansion of $G(\beta)$ and solving for $\hat{V}(\hat{\beta})$. The result is the following: 

$$\hat{V}(\hat{\beta}) = \left[I(\beta)\right]^{-1} = -H^{-1}\left[\hat{V}\{G(\hat{\beta})\}\right]-H^{-1}$$ 

for $\beta=\hat{\beta}$. The **hessian** is denoted as $H$ and it is a $(k + 1) \times (k + 1)$ matrix of second-derivatives of the log-likelihood 

$$H = \left(\frac{\partial^2 \ln L(\beta|y_j, x_j)}{\partial \hat{\beta} \partial \hat{\beta}'}\right)$$ 

where $k$ is number of independent variables except the constant. This method is referred to as the *sandwich estimator*, since the variance of $G(\beta)$ is the meat between the inverse hessians. The representative function is the `sandwich_step` in `robust_se`. 

(2) The `observed_information_matrix`, which is the inverse of the Hessian matrix. This returns the variance-covariance matrix of the paramters and square-rooting the diagonal of variances, returns the standard errors. 

(3) `outer_product_of_gradients`. Taking the inverse of the product of two jacobian matrices returns the variance-covariance matrix for the parameters and likewise, returns the standard errors from squarerooting the diagonal. This is the least most computationally demanding given second order conditions can be difficult to derive and the jacobian has already been calculated. The estimator is $$\left[I(\beta)\right]^{-1} = \left[\sum^n_{j=1} g_j'g_j\right]^{-1} $$

In [1]:
def observed_information_matrix(hess):
    """Observed information matrix or BHHH estimator.

    Args:
        hess (np.array): "hessian" - a k + 1 x k + 1-dimensional array of
            second derivatives of the pseudo-log-likelihood function w.r.t.
            the parameters
    Returns:
        oim_se (np.array): a 1d array of k + 1 standard errors
        oim_var (np.array): 2d variance-covariance matrix

    """
    hess = -hess.copy()
    oim_var = np.linalg.inv(hess)
    oim_se = np.sqrt(np.diag(oim_var))
    return oim_se, oim_var


def outer_product_of_gradients(jac):
    """Outer product of gradients estimator.

    Args:
        jac (np.array): "jacobian" - an n x k + 1-dimensional array of first
            derivatives of the pseudo-log-likelihood function w.r.t. the parameters

    Returns:
        opg_se (np.array): a 1d array of k + 1 standard errors
        opg_var (np.array): 2d variance-covariance matrix

    """
    opg_var = np.linalg.inv(np.dot(jac.T, jac))
    opg_se = np.sqrt(np.diag(opg_var))
    return opg_se, opg_var


def sandwich_step(hess, meat):
    """The sandwich estimator for variance estimation.

    Args:
        hess (np.array): "hessian" - a k + 1 x k + 1-dimensional array of
            second derivatives of the pseudo-log-likelihood function w.r.t.
            the parameters
        meat (np.array): the variance of the total scores

    Returns:
        se (np.array): a 1d array of k + 1 standard errors
        var (np.array): 2d variance-covariance matrix

    """
    invhessian = np.linalg.inv(hess)
    var = np.dot(np.dot(invhessian, meat), invhessian)
    se = np.sqrt(np.diag(var))
    return se, var

The inner component, $\hat{V}\{G(\hat{\beta})\}$, is the variance of a sum of partial derivatives. The variance of any sum is: $$\frac{n}{n-1} \sum^{n}_{j=1}(u_j-\overline{u_j})^2$$ The mean of the score vector is zero, thus $\overline{u_j} =0$. Thus, the robust variance estimator is: 

$$\hat{V}(\hat{\beta}) = \left[-H^{-1}\left(\frac{n}{n-1} \sum^{n}_{j=1}(u^{'}_ju_j)\right)-H^{-1}\right]$$

The use of this variance estimator is justified for independent observations. In the case where a group of observations are correlated, we have to adjust the estimator to take the sum over clusters compared to taking sums over individual observations. 

In [ ]:
def robust_se(jac, hess):
    """Robust standard errors.

    Args:
        jac (np.array): "jacobian" - an n x k + 1-dimensional array of first
            derivatives of the pseudo-log-likelihood function w.r.t. the parameters
        hess (np.array): "hessian" - a k + 1 x k + 1-dimensional array of second derivatives
            of the pseudo-log-likelihood function w.r.t. the parameters

    Returns:
        se (np.array): a 1d array of k + 1 standard errors
        var (np.array): 2d variance-covariance matrix

    """
    sum_scores = np.dot((jac).T, jac)
    meat = (len(jac) / (len(jac) - 1)) * sum_scores
    se, var = sandwich_step(hess, meat)
    return se, var

The cluster-robust variance is: 

$$\hat{V}(\hat{\beta}) = \left[-H^{-1}\left(\frac{n_c}{n_c-1} \sum^{n_c}_{i=1}\left(\sum_{j\in C_i}u_{j}\right)^{'}\left(\sum_{j\in C_i}u_{j}\right)\right)-H^{-1}\right]$$ 

where $n_c$ refers to the number of clusters and $\sum_{j\in C_i}u_j$ is the sum of the scores in cluster $j$. 

In [ ]:
def clustering(design_options, jac):
    """Variance estimation for each cluster.

    The function takes the sum of the jacobian observations for each cluster.
    The result is the meat of the sandwich estimator.

    Args:
        design_options (pd.DataFrame): dataframe containing psu, stratum,
            population/design weight and/or a finite population corrector (fpc)
        jac (np.array): "jacobian" - an n x k + 1-dimensional array of first
            derivatives of the pseudo-log-likelihood function w.r.t. the parameters

    Returns:
        cluster_meat (np.array): 2d square array of length k + 1. Variance of
            the likelihood equation (Pg.557, 14-10, Greene 7th edition)

    """

    list_of_clusters = design_options["psu"].unique()
    meat = np.zeros([len(jac[0, :]), len(jac[0, :])])
    for psu in list_of_clusters:
        psu_scores = jac[design_options["psu"] == psu]
        psu_scores_sum = psu_scores.sum(axis=0)
        meat += np.dot(psu_scores_sum[:, None], psu_scores_sum[:, None].T)
    cluster_meat = len(list_of_clusters) / (len(list_of_clusters) - 1) * meat
    return cluster_meat


def cluster_robust_se(jac, hess, design_options):
    """Cluster robust standard errors.

    A cluster is a group of observations that correlate amongst each other,
    but not between groups. Each cluster is seen as independent. As the number
    of clusters increase, the standard errors approach robust standard errors.

    Args:
        jac (np.array): "jacobian" - an n x k + 1-dimensional array of first
            derivatives of the pseudo-log-likelihood function w.r.t. the parameters
        hess (np.array): "hessian" - a k + 1 x k + 1-dimensional array of
            second derivatives of the pseudo-log-likelihood function w.r.t.
            the parameters
    Returns:
        cluster_robust_se (np.array): a 1d array of k + 1 standard errors
        cluster_robust_var (np.array): 2d variance-covariance matrix

    """
    cluster_meat = clustering(design_options, jac)
    cluster_robust_se, cluster_robust_var = sandwich_step(hess, cluster_meat)
    return cluster_robust_se, cluster_robust_var

`likelihood_inference` also allows for strata. Strata (plural) are constructed using information (e.g., age, income, gender). For illustration, suppose I have two age categories, "old" and "young" and two income categories, "low-income" and "high-income". The strata given this information would be the combinations: (old, low-income), (old, high-income), (young, low-income), and (young, high-income). Stratification is used as a variance reduction method. Observe this split of the data as defining sub-populations from the larger population. Strata may contain independent observations or clusters (psu's). The strata-robust variance is: $$\hat{V}(\hat{\beta}) = \left[-H^{-1}\left(\sum^L_{h=1}(1 - f_h)\frac{n_c}{n_c-1} \sum^{n_c}_{j=1}\left(\sum_{j\in C_i}u_j - \overline{u_j}\right)^{'}\left(\sum_{j\in C_i}u_j - \overline{u_j}\right)\right)-H^{-1}\right]$$ where $L$ refers to how many stratum, $h$, there are, $n_c$ refers to the number of clusters in $h$ and $\sum_{j\in C_i}u_j$ is the sum of the scores in cluster $j$ for strata $h$. $\overline{u_j}$ does not equal zero here, since we take the mean of the jacobian vector for that strata; thus, a subset of the data. Otherwise, for the entire sample, the mean of the jacobian vector is zero, as in cluster robust. The calculation is roughly the same as `cluster_robust_se`, except the calculation is done for each strata. $(1 - f_h)$ is the finite population corrector, where its default value is $1$ ($f_h = 0$). 

The process is the following:

1)  Each cluster or psu contains at least one observation, but typically more than one. We first take the sum of the jacobian observations in that cluster. If the cluster has 10 observations, we take the sum of those 10 jacobian observations, returning one row. We do this for all clusters in strata $h$ and stack the sums, to create $n_c \times (k + 1)$ matrix. 

2) To calculate $\overline{u_j}$, we take the mean of each column of $u_j$ and subtract it from that same column.

3) Then, we transpose the matrix and multiply it by the untransposed version of it. This represents $\sum^{n_c}_{j=1}\left(\sum_{j\in C_i}u_j - \overline{u_j}\right)^{'}\left(\sum_{j\in C_i}u_j - \overline{u_j}\right)$. 

4) We multiply this sum by $\frac{n_c}{n_c-1}$, where $n_c$ is the number of clusters in that particular strata. We also multiply the sum by the finite population corrector if provided. 

5) Repeat this process for all strata and sum all the resulting matrices. The size of the matrix for each iteration in the sum is always $(k + 1)\times (k + 1)$. This is of course expected because the result is the variance-covariance matrix. 

In [ ]:
def stratification(design_options, jac):
    """Variance estimatio for each strata stratum.

    The function takes the sum of the jacobian observations for each cluster
    within strata. The result is the meat of the sandwich estimator.

    Args:
        design_options (pd.DataFrame): dataframe containing psu, stratum,
            population/design weight and/or a finite population corrector (fpc)
        jac (np.array): "jacobian" - an n x k + 1-dimensional array of first
            derivatives of the pseudo-log-likelihood function w.r.t. the parameters

    Returns:
        strata_meat (np.array): 2d square array of length k + 1. Variance of
        the likelihood equation

    """
    n_params = len(jac[0, :])
    stratum_col = design_options["strata"]
    # Stratification does not require clusters
    if "psu" not in design_options:
        design_options["psu"] = design_options.index
    else:
        pass
    psu_col = design_options["psu"]
    strata_meat = np.zeros([n_params, n_params])
    # Variance estimation per stratum
    for stratum in stratum_col.unique():
        psu_in_strata = psu_col[stratum_col == stratum].unique()
        psu_jac = np.zeros([n_params])
        if "fpc" in design_options:
            fpc = design_options["fpc"][stratum_col == stratum].unique()
        else:
            fpc = 1
        # psu_jac stacks the sum of the observations for each cluster.
        for psu in psu_in_strata:
            psu_jac = np.vstack([psu_jac, np.sum(jac[psu_col == psu], axis=0)])
        psu_jac_mean = np.sum(psu_jac, axis=0) / len(psu_in_strata)
        if len(psu_in_strata) > 1:
            mid_step = np.dot(
                (psu_jac[1:] - psu_jac_mean).T, (psu_jac[1:] - psu_jac_mean)
            )
            strata_meat += (
                fpc * (len(psu_in_strata) / (len(psu_in_strata) - 1)) * mid_step
            )
        # Apply "grand-mean" method for single unit stratum
        elif len(psu_in_strata) == 1:
            strata_meat += fpc * np.dot(psu_jac[1:].T, psu_jac[1:])

    return strata_meat


def strata_robust_se(jac, hess, design_options):
    """Cluster robust standard errors.

    A stratum is a group of observations that share common information. Each
    stratum can be constructed based on age, gender, education, region, etc.
    The function runs the same formulation for cluster_robust_se for each
    stratum and returns the sum. Each stratum contain primary sampling units
    (psu) or clusters. If observations are independent, but wish to have to
    strata, make the psu column take the values of the index.

    Args:
        jac (np.array): "jacobian" - an n x k + 1-dimensional array of first
            derivatives of the pseudo-log-likelihood function w.r.t. the parameters
        hess (np.array): "hessian" - a k + 1 x k + 1-dimensional array of
            second derivatives of the pseudo-log-likelihood function w.r.t.
            the parameters
        design_options (pd.DataFrame): dataframe containing psu, stratum,
            population/design weight and/or a finite population corrector (fpc)

    Returns:
        strata_robust_se (np.array): a 1d array of k + 1 standard errors
        strata_robust_var (np.array): 2d variance-covariance matrix

    """
    strata_meat = stratification(design_options, jac)
    strata_robust_se, strata_robust_var = sandwich_step(hess, strata_meat)
    return strata_robust_se, strata_robust_var

### References 

[1] Stata 13 Base Reference Manual. Section: robust. College Station, TX: Stata Press. StataCorp. 2013.

[2] Greene, William H. Econometric analysis. Pearson Education India, 2003.

[3] Taboga, Marco. Lectures on Probability Theory and Mathematical Statistics - 3rd Edition. 